#  <center> ADQUISICION Y PREPROCESAMIENTO <center>
##  <center> RETINA <center>

## CONTEXTO

### Sobre el proyecto
Nuestro proyecto consiste en detectar a partir de la videocaptura de un rostro la ubicacion de los ojos, hacerle un seguimiento y posteriormente evaluar diferentes acciones que el usuario pueda realizar a traves de los movimientos que realice con los ojos. Es importante mencionar que la videocaptura se realizara por medio de un celular o camara para computadores por lo que utilizamos como referencia una camara generica Huawei con las siguientes caracteristicas:

8 MP, f/2.0

Estos parametros son invariables y debemos trabajar con ellos por medio de preprocesamiento para poder llegar a una etapa de segmentacion exitosa a futuro.

### Sobre la adquisicion y el preprocesmiento
Idealmente nosotros haremos en una etapa posterior un proceso de Machine Learning que nos permita separar los ojos del fondo y todos los elementos externos que puedan generar ruido, por esta razon partiremos para la adquisicion y el procesamiento de la base anteriormente mensionada y nos enfocaremos en tratar imagenes y videos que contengan unicamente el ojo y sus alrederoros mas cercanos para determinar los mejores filtros y funciones a utilizar. 




## ADQUISICION

### Metodo de adquisicion

La adquisicon sera un proceso sencillo puesto que no podemos reparar en las condiciones ambientales, las caracteristicas de los elementos de obtencion o la calidad final de la imagen, es por esta razon que hemos obtado por un proceso de Machine Learning para segmentar de la imagen completa solamente lo que nos interesa, la adquisicion se hara atraves de camaras de celulares o camaras de portatiles o usb, estas camaras tienen una gran variedad de caracteristicas distintas, estas camaras nos entregaran video a que trataremos a 30fps frame por frame. 

Las imagenes que obtendremos del rostro y los ojos sera paralela al rostro como se muestra en la figura 1. 

<center><i>Figura 1. Esquema de imagenes a obtener.</i></center>
<img src="entrega/1.png" alt="1" style="width:700px">

y una vez se haya aplicado el proceso de Machine Learning y se obtenga de la imagen solo aquello que nos interesa tendremos una imagen del estilo de la figura 2.

<center><i>Figura 2. Segmentacion por Machine Learning.</i></center>
<img src="entrega/2.png" alt="2" style="width:300px">



In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

### Adquisicion de las imagenes e importacion a python
#### Función de captura
<p>Esta función captura la webcam del computador y posteriormente se le aplica un filtro a cada frame del video capturado.</p>

In [2]:
def captureAndApplyFilter(f, fparams=[]):
    
    # Get a reference to webcam #0 (the default one)
    video_capture = cv2.VideoCapture(0)

    while True:
        # Grab a single frame of video
        ret, frame = video_capture.read()
        frame = cv2.cvtColor( frame, cv2.COLOR_BGR2GRAY )
    
        # Apply a filter to the frame
        res = f(frame,*fparams)

        cv2.imshow('Video', res)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release handle to the webcam
    video_capture.release()
    cv2.destroyAllWindows()
    print('Released Video Resource')

## PREPROCESAMIENTO

En el prepocesamiento utilizamos distintos tipos de filtros con el fin de resaltar el iris para que en una posterior etapa de segmentacion sea mucho mas facil de reconocer.

### Parámetros
<ul>
<li>Función filtro que se le va a aplicar a cada frame.</li>
<li>Array, en orden, con los parámetros de la función filtro.</li>
</ul>

### Funciones filtro
<p>Funciones que reciben una imagen y devuelven una imagén modificada.</p>

<ul>
<li>Sin filtro</li>
<li>Negativo</li>
<li>Raiz con MedianBlur</li>
<li>Thresholded</li>
</ul>

In [1]:
def nonFilter(img): return img

def negative(img):
    img1_subtract= np.zeros(img.shape, np.uint8)
    img1_subtract = cv2.subtract(255, img)
    img1_subtract = cv2.subtract(img1_subtract,100)
    return img1_subtract


def non_linear_rootMod(img, a, b):
    
    img_copy = img.copy().astype(np.float32)/255.0
    res_a = cv2.pow(img_copy,0.5)
    res_a = cv2.multiply(res_a, a)
    res = cv2.add(res_a,b)
    
    res[res<0] = 0
    res = res*255.0
    res[res>255] = 255
    
    res = res.astype(np.uint8)
    res = cv2.medianBlur(res,5)
    
    return res

def thresholdImg(img):
#     th, thresholded = cv2.threshold(img,200,255,cv2.THRESH_BINARY)
#     thresholded = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    thresholded = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
    return thresholded

### Sin filtro

In [ ]:
# Normal
captureAndApplyFilter(nonFilter)

### Negativo

In [ ]:
# Negative
captureAndApplyFilter(negative)

### Raiz con MedianBlur

In [ ]:
# Raiz con MedianBlur
a = 1.3
b = -0.1
params = [a,b]
captureAndApplyFilter(non_linear_rootMod,params)

### Thresholded

In [ ]:
# Threshold
captureAndApplyFilter(thresholdImg)